# Práctica 4 - FastSLAM y Planeamiento de Trayectoria

**Profesor**: Prof. Dr. Ignacio Mas

**Fecha límite de entrega**: 24/11/2024, 23:59hs

---

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.colors import Normalize

from probabilistic_robotics.localization import KalmanFilter
from probabilistic_robotics.mapping import GridMapping1D


%load_ext autoreload
%autoreload 2

## 1. Implementación de algoritmo FASTSLAM

En este ejercicio se implementará el algoritmo FASTSLAM basado en landmarks. Se asume que los landmarks son identificables por lo que el problema de asociación de datos está resuelto.

### 1.1. Paso de corrección FASTSLAM

Implementar el paso de corrección en la función correction `correction_step(...)`. Asumir que el ruido de medición está caracterizado por la matriz diagonal cuadrada de $2\times 2$ $Q_t$:
$$
\begin{equation}
Q_t = 
\begin{bmatrix} 
0.1 & 0.0 \\
0.0 & 0.1
\end{bmatrix}
\end{equation}
$$

**Respuesta**: Se ejecutó el script `fastslam.py` que usa la clase `FastSLAM` para implementar el algoritmo. La clase contiene el método `update` que llama a `prediction_step(...)` y `correction_step(...)` para actualizar el estado del filtro. La función `correction_step(...)` se implementó para actualizar el estado del filtro de partículas en función de las mediciones de los sensores. El video obtenido parece mostrar un comportamiento razonable del filtro al comienzo de la simulación, pero luego comienza a diverger.

## 2. Planeamiento de caminos

Los algoritmos de búsqueda en grafos como Dijkstra o A* pueden ser usados para planear caminos en grafos desde un lugar de inicio hasta un objetivo. Si las celdas de un mapa de grilla se representan como nodos conectados con sus celdas vecinas, estos algoritmos pueden aplicarse directamente para realizar planeamiento para robots. Para este ejercicio, consideramos las 8 celdas vecinas de una celda $\langle x, y \rangle$, que se definen como las celdas adyacentes a $\langle x, y \rangle$ horizontalmente, verticalmente y en diagonal. 

El archivo incluido contiene una implementación de planeamiento en 2-D basado en grafos. El script `planning framework.m` contiene la parte principal del algoritmo y es el que debe ejecutarse. Este archivo no necesita ser modificado, pero es aconsejable entender lo que hace. Los ejercicios de esta sección se realizan implementando las funciones vacías que acompañan al script principal.

### 2.1. Algoritmo de Dijkstra

El algoritmo de Dijkstra se usa para calcular caminos de costo mínimo dentro de un grafo. Durante la búsqueda, siempre se busca el nodo del grafo con el menor costo desde el punto de inicio y se agregan los nodos vecinos al grafo de búsqueda.

1. Sea $M(x, y)$ un mapa de grilla de ocupación. Durante la búsqueda, las celdas se conectan con sus celdas vecinas para construir el grafo de búsqueda. Completar la función `neighbors` provista que define los vecinos de una celda. La función toma como entrada las coordenadas de una celda y el tamaño del mapa, y devuelve un vector de $n \times 2$ con las coordenadas de sus celdas vecinas, teniendo en cuenta los límites del mapa.

2. Implementar una función para los costos de un arco entre nodos que permita planear caminos de mínima longitud y libre de colisiones. Considerar la celda como un obstáculo si su probabilidad de ocupación supera cierto umbral. ¿Qué umbral se debería elegir? Implementar la función `edge_cost`.

3. Incluir información de ocupación en la función de costo que permita que el algoritmo elija celdas con baja probabilidad de ocupación sobre celdas con mayor
probabilidad de ocupación.

**Respuesta**: Se implementó el algoritmo de Dijkstra en el script `planning.py` con el flag `--method dijkstra`. Se completó la función `get_neighbors` para obtener los vecinos de una celda y la función `get_edge_cost` para calcular el costo de un arco entre nodos teniendo encuenta celdas de ocupación. Ambas funciones pertenecen a la clase `Planning` que implementa el algoritmo de planeamiento. Se eligió un umbral de 0.5 para considerar una celda como obstáculo. El video `dijsktra.mp4` muestra el resultado de la planificación de caminos y la última imágene muestra el mapa con el camino planificado.

## 2.2. Algoritmo $A^*$

El algoritmo $A^*$ utiliza una heurística para realizar una búsqueda informada que resulta ser más eficiente que el algoritmo de Dijkstra.

1. ¿Qué propiedades debe tener dicha heurística para asegurar que $A^*$ es óptimo?

2. Definir una heurística para planeamiento de robots móviles en 2-D. Completar la función heuristic provista. La función toma como entrada las coordenadas de
una celda y del objetivo, y devuelve el costo estimado hasta el objetivo.

3. ¿Qué pasa si se aumenta la heurística usando $h_2$, siendo $h_2$ un múltiplo de la heurística $h$ definida en el punto anterior. Analizar el comportamiento con diferentes factores: $h_2 = a\,h,\;\; a \in \{1, 2, 5, 10\}$

**Respuesta**: La heurística debe ser admisible y consistente para asegurar que $A^*$ es óptimo. Se eligió una heurística como la suma de dos términos:
$$
h = 10 \cdot \mathrm{ProbOcupacion}(x, y) + \mathrm{DistanciaAlOptimo}(x, y)
$$
El término $\mathrm{ProbOcupacion}(x, y)$ representa la probabilidad de ocupación de la celda y el término $\mathrm{DistanciaAlOptimo}(x, y)$ representa la distancia euclidiana entre la celda y el objetivo. Se implementó la función `heuristic` en la clase `Planning` para calcular la heurística. Se analizó el comportamiento del algoritmo $A^*$ con diferentes factores multiplicativos de la heurística. Los resultados se muestran en los videos `astar_h2=1.mp4`, `astar_h2=2.mp4`, `astar_h2=5.mp4` y `astar_h2=10.mp4`.